# Exploring DEGURBA
The Degree of Urbanization methodology developed by the European Commission provides a consistent definition of urban through application of population density and total population theresholds to gridded population data.

In [ ]:
import sys
import os
import importlib
import rasterio
import rasterio.warp

import geopandas as gpd
import numpy as np

from rasterio import features
from shapely.geometry import Polygon

# Import GOST urban functions
sys.path.append("../GOST_Urban")
import src.UrbanRaster as urban

# Import raster helpers
sys.path.append("../../gostrocks/src")

import GOSTRocks.rasterMisc as rMisc
from GOSTRocks.misc import tPrint

: 

In [21]:
global_bounds_adm1 = "/home/public/Data/GLOBAL/ADMIN/Admin1_Polys.shp"
iso3 = "GHA"
output_folder = "/home/wb411133/temp/%s_URBAN_DATA" % iso3
pop_layer = os.path.join(output_folder, "FINAL_STANDARD", "%s_GHS.tif" % iso3)
temp_pop_file = os.path.join(output_folder, "GHS_TEMP.tif")
deg_file = os.path.join(output_folder, "FINAL_STANDARD", "GHS_DEG.tif")
adm1_file = os.path.join(output_folder, "ADM1.shp")

In [25]:
inG1 = gpd.read_file(global_bounds_adm1)
inD1 = inG1.loc[inG1["ISO3"] == iso3]
inD1 = inD1.to_crs({"init": "epsg:4326"})
if not os.path.exists(adm1_file):
    inD1.to_file(adm1_file)

if not os.path.exists(temp_pop_file):
    inR = rasterio.open(pop_layer)
    selD = inD1.loc[inD1["WB_ADM1_NA"] == "Central"]
    rMisc.clipRaster(inR, selD, temp_pop_file)

national_ghs_1k = os.path.join(output_folder, "GHS_POP_1k.tif")
if not os.path.exists(national_ghs_1k):
    inR = rasterio.open(
        "/home/public/Data/GLOBAL/Population/GHS/GHS_POP_E2015_GLOBE_R2019A_54009_1K_V1_0.tif"
    )
    inD = gpd.read_file(os.path.join(output_folder, "FINAL_STANDARD", "GHA_ADMIN.shp"))
    if inD.crs != inR.crs:
        inD = inD.to_crs(inR.crs)
    rMisc.clipRaster(inR, inD, national_ghs_1k)

In [24]:
importlib.reload(urban)
xx = urban.urbanGriddedPop(pop_layer)
res = xx.calculateDegurba(
    urbDens=21, hdDens=(15 * 7), minPopThresh=0.5 * 7, out_raster=deg_file, verbose=True
)

11:06:51	: Smoothing Urban Clusters
11:16:55	: extracting HD clusters
11:16:58	: Creating Shape 0
11:18:26	: Creating Shape 1000
11:19:52	: Creating Shape 2000
11:21:18	: Creating Shape 3000
11:22:44	: Creating Shape 4000
11:24:11	: Creating Shape 5000
11:25:37	: Creating Shape 6000
11:27:03	: Creating Shape 7000
11:28:27	: Creating Shape 8000
11:29:26	: extracting URBAN clusters
11:30:05	: Creating Shape 9000
11:31:33	: Creating Shape 10000
11:32:58	: Creating Shape 11000
11:34:27	: Creating Shape 12000
11:35:52	: Creating Shape 13000
11:37:20	: Creating Shape 14000
11:38:45	: Creating Shape 15000
11:40:06	: Creating Shape 16000
11:41:32	: Creating Shape 17000
11:43:01	: Creating Shape 18000
11:44:25	: Creating Shape 19000
11:45:51	: Creating Shape 20000
11:47:19	: Creating Shape 21000
11:48:43	: Creating Shape 22000
11:49:58	: Creating Shape 23000
11:51:21	: Creating Shape 24000
11:52:45	: Creating Shape 25000
11:53:08	: performing distance calculations


In [26]:
importlib.reload(urban)
deg_file = national_ghs_1k.replace(".tif", "DEGURBA.tif")
xx = urban.urbanGriddedPop(national_ghs_1k)
res = xx.calculateDegurba(
    urbDens=300, hdDens=1500, minPopThresh=50, out_raster=deg_file, verbose=True
)

12:18:16	: Smoothing Urban Clusters
12:18:54	: extracting HD clusters
12:18:54	: Creating Shape 0
12:18:58	: Creating Shape 1000
12:18:59	: extracting URBAN clusters
12:19:02	: Creating Shape 2000
12:19:06	: Creating Shape 3000
12:19:10	: Creating Shape 4000
12:19:14	: Creating Shape 5000
12:19:15	: performing distance calculations


In [17]:
importlib.reload(urban)
temp_pop_file = os.path.join(
    output_folder, "GHS_POP_E2015_GLOBE_R2019A_54009_1K_V1_0_22_3.tif"
)
deg_file = temp_pop_file.replace(".tif", "DEGURBA.tif")
xx = urban.urbanGriddedPop(temp_pop_file)  # (pop_layer)
res = xx.calculateDegurba(
    urbDens=300, hdDens=1500, minPopThresh=50, out_raster=deg_file, verbose=True
)

09:13:24	: Smoothing Urban Clusters
09:15:11	: extracting HD clusters
09:15:11	: Creating Shape 0
09:15:13	: extracting URBAN clusters
09:15:18	: performing distance calculations


In [15]:
features.shapes?

# Plotting

In [ ]:
import os
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.colors as colors

In [ ]:
cmap = ListedColormap(
    ["lightgreen", "green", "darkgreen", "yellow", "brown", "black", "red"]
)
norm = colors.BoundaryNorm([10, 11, 12, 13, 21, 22, 23, 30], 7)
fig, ax = plt.subplots(figsize=(10, 5))
ax.imshow(hd[350:450, 500:650], cmap=cmap, norm=norm)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.imshow(urb[350:450, 500:650], cmap=cmap, norm=norm)  # , extent=[350, 450, 650, 500])
plt.show()

In [ ]:
ax.imshow?

# DEBUGGING

In [ ]:
import geojson
import json

from shapely.geometry import shape
from rasterio.features import rasterize

urbDens = 3 * 7
hdDens = 15 * 7
urbThresh = 5000
hdThresh = 50000
minPopThresh = 0.5 * 7
out_raster = deg_file
print_message = ""
verbose = False

In [ ]:
xx = urban.urbanGriddedPop(temp_pop_file)
popRaster = xx.inR
data = popRaster.read()
urban_raster = data * 0
final_raster = data[0, :, :] * 0 + 11

urban_raster[np.where(data > hdDens)] = 30

idx = 0
allFeatures = []
# Analyze the high density shapes
for cShape, value in features.shapes(urban_raster, transform=popRaster.transform):
    if idx % 1000 == 0 and verbose:
        tPrint("%s: Creating Shape %s" % (print_message, idx))
    idx = idx + 1
    if value > 0:
        if value == 30:  # only smooth the HD density shapes
            origShape = cShape
            xx = shape(cShape)
            xx = Polygon(xx.exterior)
            cShape = xx.__geo_interface__
        # If the shape is urban, claculate total pop
        mask = rasterize(
            [(cShape, 0)],
            out_shape=data[0, :, :].shape,
            fill=1,
            transform=popRaster.transform,
        )
        inData = np.ma.array(data=data, mask=mask.astype(bool))
        pop = np.nansum(inData)

        val = 0
        if pop > 5000:
            ### TODO - if the totalpop is < 50k, may need to unsmooth the shape
            val = 23
        if pop > 50000:
            val = 30

        # Burn value into the final raster
        mask = (mask ^ 1) * val
        yy = np.dstack([final_raster, mask])
        final_raster = np.amax(yy, axis=2)
        allFeatures.append([idx, pop, val, shape(geojson.loads(json.dumps(cShape)))])
HD_raster = final_raster

In [ ]:
unique_elements, counts_elements = np.unique(HD_raster, return_counts=True)
print(np.asarray((unique_elements, counts_elements)))

In [ ]:
final_raster = data[0, :, :] * 0
yy = np.dstack([final_raster, mask])
final_raster = np.amax(yy, axis=2)

In [ ]:
mask = rasterize(
    [(cShape, 0)], out_shape=data[0, :, :].shape, fill=1, transform=popRaster.transform
)

In [ ]:
tempR = rasterio.open(temp_pop_file)
tempR.shape

In [ ]:
tempR.meta